# Preparing a study for SPM12w

## Study Directory Structure

SPM12w expects a BIDS-compliant dataset. That said, there are several ways to organize BIDS data that are all euqally compliant. The current version of SPM12w expects the BIDS dataset to be organized as outlined below. 

```{note} 
The requirement that the bids-compliant datasets used in spm12w follow the conventions below does mean that users will have to buy into our way of organizing bids data and specifying task __.tsv__ files. 

Users taunting SPM12w with idiosyncratic file structures do so at their own peril. 
```

The following is a minimal example of a new BIDS study ready to be processed with SPM12:

```bash
tutorial_data/
├── derivatives
│   └── wanger.1174
│       └── code
├── raw_data
│   └── sub-s01
│       ├── anat
│       └── func
└── sourcedata
    ├── code
    ├── dicom
    └── tsvfiles
```

```{admonition} Click here for a full description of each directory's contents
:class: dropdown, tip
* tutorial_data: __Name of the study directory__
* ./derivatives: __Location of BIDS derivatives (analyses and code)__
* ./derivatives/wagner.1174: __username. All derivatives are seperated according to user__
* ./derivatives/wagner.1174/code: __Location of spm12w parameter files and any other code__
* ./raw_data: __Location of bids-compliant dataset converted from files in sourcedata__
* ./raw_data/sub-s01: __Directory name of a single subject__
* ./raw_data/sub-s01/anat: __Location of anatomical data__
* ./raw_data/sub-s01/func: __Location of functional data__
* ./raw_data/dwi: __Location of dwi data__
* ./sourcedata: __Location of sourcedata__
* ./sourcedata/dicom: __Location of dicom archived from the scanner__
* ./sourcedata/code: __Optional location of code to be run after dicom conversion__
* ./sourcedata/tsviles: __Optional location of csv files to help create bids-compatible tsv files__

```

## Conversion to BIDS using SPM12w

To begin using SPM12, it is necessary to have BIDS compliant directory stored under the study's rawdata directory (see above). The easiest way to generate one is to use __spm12w_prepare.m__. This script will read in DICOM or PAR/REC data stored under sourcedata and convert these to BIDS using the matlab based DICOM and PAR/REC converter [__dicm2nii__](https://github.com/xiangruili/dicm2nii). Essentially, this script wraps __dicm2nii.m__ to facilitate the conversion to BIDS.

When run from the root of the study directory (e.g., tutorial_data), __spm12w_prepare.m__ will look for a dicom or par/rec directory and provide a list of archives for conversion (either tar.gz or zip). For __spm12w_prepare.m__ to determine the appropriate naming for each subject, the archives must be named according to the following convention. 


___parrec_tutorial01_9001DW_24feb01_s0018.zip___

````{tab-set}

```{tab-item} dicom
Sourcedata format, can be either dicom or parrec
```

```{tab-item} tutorial01
BIDS subject ID for current study
```

```{tab-item} 9001DW
Scanner ID set by imaging center
```

```{tab-item} 24feb01
Date of scan
```

```{tab-item} s0018
Unique subject ID in thelab database
```
````   

The prepare script has several optional arguments. Below is the help section of __spm12w_prepare.m__

```octave
spm12w_prepare(scannerid, sid, bidsgui, ptsv)
%
% Inputs
% ------
% scannerid: ScannerID tag or Filename of archived raw scanner data to be
% converted to BIDS and "prepared" for spm12w. If left
% unspecified, then spm12w_prepare will open a window allowing
% the user to select the files to prepare based on the
% information found in the fields of the matching archive
% filename. If specifying the full filename it can be specified
% with or without extension.
%
% sid: sid name that the raw scanner data will be renamed to during
% conversion. This corresponds to the subject identifier to be
% used for this study (e.g., s01, s02, s03, etc.). This will
% be converted to bids format (e.g., sub-s01). If you secify
% subject IDs in bids format then spm12w_prepare will not add
% the 'sub-' prefix.
%
% bidsgui: Switch (true or false) to force the bidsGUI during conversion.
% Useful when needing to fix previously set preferences for bids
% conversion in dcm2nii, such as when you previously skipped a
% series that you now wish to keep. (default = true).
%
% ptsv: Switch (true or false) to enable creation of the
% participants.tsv file. If false, the data extracted from the
% dicom headers (i.e., weight, height) can still be found in
% the log files stored under sourcedata. (default = true)
```

Because the dicom conversion tool we wrap (dicm2nii) stores BIDS dicom series naming preferences in a hidden Matlab preferences file, it can be difficult to recover from mistakenly naming a run or if you decide to use different BIDS task names at a later date. As the tool will only open the GUI for DICOM series names it does not recognize, we've added a __bidsgui__ switch. When set to true, it will force open the bids GUI allowing the user to edit names and skip any aborted runs etc. We've found that manual intervention is almost always warranted (i.e., to skip aborted runs or fix naming issues) and so the default for bidsgui is true.

The workflow for spm12w_prepare is as follows:
1. Locate full path to the dicom archive file based on scannerID or filename
1. Convert to bids compliant nifti via dicm2nii.
1. Copy to a BIDS compliant rawdata directory renaming scannerID to the provided subject ID.
1. Edit participants.tsv file to reflect the appropriate sid (default: True)
1. Create modality invariant BIDS files, fix any json fields. 
1. Output logs and other files to sourcedata/dicom/logs

### Below are some examples of different uses of spm12w_prepare

```octave
% Examples:
%
% Without arguments, spm12w will prompt the user to select scanner archive
% files for "preparing".
%
% >> spm12w_prepare
%
% With arguments, spm12w prepare the specified archive files. This case is
% useful if you want to avoid using the file selector gui.
%
% >> spm12w_prepare('scannerid',{'4077DW','4099DW'}, "sid', {'s01','s02'})
% >> spm12w_prepare('bidsgui', false, 'ptsv', false)
```

The image below shows the conversion for a study containing multiple fMRI runs, anatomy, DWI, fieldmaps, and some additional files collected by the scanner that we don't need. When using spm12w_prepare and dicm2nii, it's important to get the modality and name correct in order for the bids conversion to be successful. Pay close attention to the names for the different sequence types and the leading zeros. If using spm12w_prepare.m for conversion, the subject name does not need to be changed as this will be done post-conversion. 

In this image you'll see the formatting for _field maps, bold images and their associated sbref, anatomy and dwi_. In addition, notice the two aborted runs that were included in the export are skipped during conversion. 

```{figure} ../assets/ch2_prepare.png
---
width: 550px
name: directive-fig
---
spm12w_prepare: example of the scanner -> bids mapping for a complicated case
```

## Creating the participant and events tsv files using SPM12w

Following the conversion of data to BIDS format, a few additional steps are needed to make the data BIDS compliant in such a way that it will pass BIDS validation [https://bids-standard.github.io/bids-validator/](https://bids-standard.github.io/bids-validator/). Firstly, the BIDS validator expects a BIDS compliant participant.tsv and json file that contains information about each participant. Although this can be anything, typically we would have age, gender, ethnicity and various individual difference scales. In addition, the bids validator expects every functional run to have accompanying BIDS compliant events.tsv and json file specifying all the information needed to model the task.

spm12w provides a utility __spm12w_tsvmaker.m__ to help create and place these _.tsv_ and _.json_ files in the appropriate locations. This utility requires that you first create template files that are located in _sourcedata/tsvfiles_. For an example of how these files are formatted, please see the tutorial dataset and the tutorial section of this manual.

The tsvmaker script has several optional arguments. Below is the help section of __spm12w_tsvmaker.m__.

```octave
spm12w_tsvmaker(sids, csvfile)
%
% Input
% -----
% sids: Cell array of subject ids. If left unspecified, a dialog box will
% appear asking the user to select subjects. If the special keyword
% 'allsids' is provided then all subjects in the rawdata directory
% will be selected.
%
% csvfile: path to a participant or task specific csv file with 1 row of
% headers.
%
% This function takes either a participants_tsv.csv file or a
% task-XXXX_tsv.csv and outputs BIDS formated tsv files in the appropriate
% locations.
%
% When using to generate task tsv files:
%
% If task.tsv file (i.e., task-tutorial_events_tsv.json)
% the file should have one row per condition with columns
% specifying any of the following options:
% - subject-specific parametric columns
% - subject-specific behavioral response
% - subject-specific response time
%
% When using to generate participant tsv files:
% If participant.tsv file (i.e., participants_tsv.csv)
% The file should have one row per subject ID, with columns
% indicating various subject-specific data (i.e., age, height
% scale measures).
```